### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1378]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [1379]:
complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [1380]:
#total number of schools
school_number = len(complete_df["school_name"].unique())
#total number of students
total_students = sum(complete_df["size"])
#total budget
total_budget = sum(complete_df["budget"])


In [1381]:
#average math score
average_math=complete_df["math_score"].mean()
#average reading score
average_reading= complete_df['reading_score'].mean()

In [1382]:
#Calculate the percentage of students with a passing math score (70 or greater)

percent_math= (complete_df[complete_df['math_score'] >= 70].math_score.count()) / total_students

#Calculate the percentage of students with a passing reading score (70 or greater)

percent_read= (complete_df[complete_df['reading_score'] >= 70].reading_score.count()) / total_students

#Calculate the percentage of students who passed math and reading (% Overall Passing)
pass_overal= complete_df[(complete_df["reading_score"]>=70) & (complete_df["math_score"]>=70)].math_score.count()
percent_pass= (pass_overal)/total_students * 100

district_summary = pd.DataFrame ({"Total Schools": [school_number],
                                "Total Number of Students": [total_students], 
                                "Total Budget": [total_budget], 
                                "Average Math Score": [average_math], 
                                "Average Reading Score": [average_reading], 
                                "Percentage of Passing Math": [percent_math],
                                "Percentage of Passing Reading": [percent_read], 
                                "Percentage of Passing": [percent_pass] })

#cleaning up total budget
district_summary['Total Budget'] = district_summary.apply(lambda x: "${:,.2f}".format(x['Total Budget']), axis=1)
district_summary['Total Number of Students'] = district_summary.apply(lambda x: "{:,.0f}".format(x['Total Number of Students']), axis=1)


district_summary.head()

,Total Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,Percentage of Passing Math,Percentage of Passing Reading,Percentage of Passing
0,15,"130,551,930","$82,932,329,558.00",78.985371,81.87784,0.000225,0.000257,0.019554


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [1383]:
school_df = complete_df.groupby('school_name')

school = complete_df['school_name'].sort_values().unique()

school_type = complete_df.set_index('type')

students_per_school= complete_df['Student ID'].count()

school_budget = list(school_df['budget'].mean())

student_budget = school_budget/students_per_school 

avg_math = school_df["math_score"].mean()

avg_read =school_df["reading_score"].mean()

pass_math = complete_df[complete_df["math_score"] >=70].groupby('school_name')['Student ID'].count()
school_percent_math = pass_math/ students_per_school *100

pass_read = complete_df[complete_df["reading_score"] >=70].groupby('school_name')['Student ID'].count()/students_per_school
school_percent_read = pass_read *100

pass_overal= complete_df[(complete_df["reading_score"]>=70) & (complete_df["math_score"]>=70)].groupby('school_name')['Student ID'].count()/students_per_school
school_percent_pass= pass_overal * 100

In [1384]:
#dataframe for school summary
school_summary = pd.DataFrame ({"School": [school],
                                "School Type": [school_type],
                                "Number of Students": [students_per_school], 
                                "School Budget": [school_budget], 
                                "Budget Per Student": [student_budget],
                                "Average Math Score": [avg_math], 
                                "Average Reading Score": [avg_read],
                                "Percentage of Passing Math": [school_percent_math],
                                "Percentage of Passing Reading": [school_percent_read], 
                                "Percentage of Passing": [school_percent_pass] }) 

school_summary.head()

,School,School Type,Number of Students,School Budget,Budget Per Student,Average Math Score,Average Reading Score,Percentage of Passing Math,Percentage of Passing Reading,Percentage of Passing
0,"[Bailey High School, Cabrera High School, Figu...",Student ID student_name gender...,39170,"[3124928, 1081356, 1884411, 1763916, 917500, 3...","[79.77860607607863, 27.606739851927497, 48.108...",school_name Bailey High School 77.048432...,school_name Bailey High School 81.033963...,school_name Bailey High School 8.470768 ...,school_name Bailey High School 10.408476...,school_name Bailey High School 6.941537 ...


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [1385]:
top_schools = school_summary.sort_values("Percentage of Passing", ascending = False)

top_schools.head(5)

,School,School Type,Number of Students,School Budget,Budget Per Student,Average Math Score,Average Reading Score,Percentage of Passing Math,Percentage of Passing Reading,Percentage of Passing
0,"[Bailey High School, Cabrera High School, Figu...",Student ID student_name gender...,39170,"[3124928, 1081356, 1884411, 1763916, 917500, 3...","[79.77860607607863, 27.606739851927497, 48.108...",school_name Bailey High School 77.048432...,school_name Bailey High School 81.033963...,school_name Bailey High School 8.470768 ...,school_name Bailey High School 10.408476...,school_name Bailey High School 6.941537 ...


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [1386]:
top_schools = school_summary.sort_values("Percentage of Passing", ascending = True)

top_schools.head(5)

,School,School Type,Number of Students,School Budget,Budget Per Student,Average Math Score,Average Reading Score,Percentage of Passing Math,Percentage of Passing Reading,Percentage of Passing
0,"[Bailey High School, Cabrera High School, Figu...",Student ID student_name gender...,39170,"[3124928, 1081356, 1884411, 1763916, 917500, 3...","[79.77860607607863, 27.606739851927497, 48.108...",school_name Bailey High School 77.048432...,school_name Bailey High School 81.033963...,school_name Bailey High School 8.470768 ...,school_name Bailey High School 10.408476...,school_name Bailey High School 6.941537 ...


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [1387]:

ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()

tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()

eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()

twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math })

math_scores = math_scores[['9th', '10th', '11th', '12th']]

math_scores.index.name = "School"

math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})


,9th,10th,11th,12th
School,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [1388]:
read_nine = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()

read_ten = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()

read_eleven = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()

read_twelve = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

reading_scores = pd.DataFrame({
        "9th": read_nine,
        "10th": read_ten,
        "11th": read_eleven,
        "12th": read_twelve })

reading_scores = reading_scores[['9th', '10th', '11th', '12th']]

reading_scores.index.name = "School"

reading_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [1389]:
bin = [0, 584.999, 614.999, 644.999, 999999]

bin_groups = ['< $585', "$585 - 614", "$615 - 644", "> $644"]

complete_df['spending_bins'] = pd.cut(complete_df['budget']/complete_df['size'], bin, labels = bin_groups)

spending= complete_df.groupby('spending_bins')

In [1390]:
avg_math = spending['math_score'].mean()

avg_read = spending['reading_score'].mean()

pass_math = complete_df[complete_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending['Student ID'].count()

pass_read = complete_df[complete_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending['Student ID'].count()

overall = complete_df[(complete_df['reading_score'] >= 70) & (complete_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/spending['Student ID'].count()

            

In [1391]:
scores_by_spending = pd.DataFrame({ "Average Math Score": [avg_math],
                                    "Average Reading Score": [avg_read],
                                    "% Passing Math": [pass_math],
                                    "% Passing Reading": [pass_read],
                                    "Overall Passing Rate": [overall]
                                })

scores_by_spending = scores_by_spending [{ 
                        "Average Math Score",
                        "Average Reading Score",
                        "% Passing Math",
                        "% Passing Reading",
                        "Overall Passing Rate" }]

scores_by_spending.index.name = "Per Student Budget"

scores_by_spending.style.format({"Average Math Score": '{:.2f}', 
                              "Average Reading Score": '{:.2f}', 
                              "% Passing Math": '{:.2f}', 
                              "% Passing Reading":'{:.2f}', 
                              "% Overall Passing": '{:.2f}'})

TypeError: unsupported format string passed to Series.__format__

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
bins = [0, 1000, 1999,5000]

group_name = [" (<1000)", " (1000-2000)" , " (2000-5000)"]

complete_df['size_bins'] = pd.cut(complete_df['size'], bins, labels = group_name)

by_size = complete_df.groupby('spending_bins')

In [ ]:
avg_math = by_size['math_score'].mean()

avg_read = by_size['reading_score'].mean()

pass_math = complete_df[complete_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_size['Student ID'].count()*100

pass_read = complete_df[complete_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_size['Student ID'].count()*100

overall = complete_df[(complete_df['reading_score'] >= 70) & (complete_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_size['Student ID'].count()*100
    

In [ ]:
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "% Overall Passing": overall})

scores_by_size.index.name = "Total Students"

scores_by_size = scores_by_size.reindex(group_name)    

scores_by_size.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2f}', 
                              '% Passing Reading':'{:.2f}', 
                              '% Overall Passing': '{:.2f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Students,,,,,
(<1000),nan,nan,nan,nan,nan
(1000-2000),nan,nan,nan,nan,nan
(2000-5000),nan,nan,nan,nan,nan


## Scores by School Type

* Perform the same operations as above, based on school type

In [ ]:
schoo_type = complete_df.groupby("type")

average_math_score = schoo_type['math_score'].mean()

average_reading_score = schoo_type['math_score'].mean()

pass_math_percent = complete_df[complete_df['math_score'] >= 70].groupby('type')['Student ID'].count()/schoo_type['Student ID'].count()*100

pass_read_percent = complete_df[complete_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/schoo_type['Student ID'].count()*100

overall = complete_df[(complete_df['reading_score'] >= 70) & (complete_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_size['Student ID'].count()*100


In [ ]:
         
scores_schoo_type = pd.DataFrame({
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    '% Passing Math': pass_math_percent,
    '% Passing Reading': pass_read_percent,
    "% Overall Passing": overall})
    

scores_schoo_type = scores_schoo_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing"
]]
scores_schoo_type.index.name = "Type of School"



scores_schoo_type.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Type of School,,,,,
$585 - 614,nan,nan,nan,nan,90.121190
$615 - 644,nan,nan,nan,nan,60.289317
< $585,nan,nan,nan,nan,90.640704
> $644,nan,nan,nan,nan,53.528791
Charter,83.406183,83.406183,93.701821,96.645891,nan
District,76.987026,76.987026,66.518387,80.905249,nan
